<a href="https://colab.research.google.com/github/YunzhenYang-collection/Expansion-Exercises-Image-Recognition/blob/main/versions/im_re_v2_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 待修正

1. 選定特定的欄位重新辨識
2. 即便辨識成功，但是辨識率太低，也告知照片調整
3. 若照片有瑕疵，能否進行修復?

### Set up

In [ ]:
!pip install -q -U google-generativeai google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client

In [ ]:
import google.generativeai as genai
from IPython.display import Markdown
# from realesrgan import RealESRGAN
import httpx
import base64
import imghdr
import re
import io
import json

import os
import cv2
from PIL import Image, ImageFilter, ImageStat
import numpy as np

In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')
from google.colab import files

In [ ]:
from google.colab import userdata
# userdata.get('GOOGLE_API_KEY')
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY2')
# 確認密鑰是否正確檢索
if GOOGLE_API_KEY:
    print("API key retrieved successfully.")
    genai.configure(api_key=GOOGLE_API_KEY)
else:
    print("Error: API key not found.")

### .env test

In [ ]:
!pip install python-dotenv

In [ ]:
from dotenv import load_dotenv
absolute_path = "/content/drive/My Drive/Colab Notebooks/CP_im_re"

os.chdir(absolute_path)
print(os.getcwd())

load_dotenv()
print(os.environ.get("IMAGE_PATH"))

## 由llm辨識圖片錯誤

In [ ]:
def encode_image(image_content):
    """Encode image content to base64"""
    return base64.b64encode(image_content).decode('utf-8')

def is_valid_for_category(image_type, category):
    """Check if image type is valid for the selected category"""
    # Add validation logic here if needed
    # For now, we'll assume all image types are valid for all categories
    return True


def error_type(image_path):
    """Check for image quality issues and call Gemini for suggestions if needed"""
    try:
        # Open image using OpenCV
        image = cv2.imread(image_path)

        # Check brightness (average pixel value)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        brightness = np.mean(gray)
        if brightness < 50:
            return {"error": "Image too dark."}
        elif brightness > 200:
            return {"error": "Image too bright."}

        # Check for blurriness (using Laplacian variance)
        variance = cv2.Laplacian(gray, cv2.CV_64F).var()
        if variance < 100:
            return {"error": "Image is blurry."}

        # Check for orientation (detect edges and estimate rotation)
        # Using Hough Transform to check for line alignment
        edges = cv2.Canny(gray, 50, 150, apertureSize=3)
        lines = cv2.HoughLinesP(edges, 1, np.pi / 180, 100, minLineLength=50, maxLineGap=10)

        if lines is not None:
            angle = np.mean([np.arctan2(line[0][3] - line[0][1], line[0][2] - line[0][0]) for line in lines])
            angle = np.degrees(angle)
            if abs(angle) > 5:  # If the angle of the lines is greater than 5 degrees
                return {"error": "Image is tilted."}

        # If no issues, return None
        return {"error": "No issues detected with image quality."}

    except Exception as e:
        # If error during processing, call Gemini for suggestions
        model = genai.GenerativeModel(model_name="gemini-1.5-pro")
        prompt = """
        請根據圖片自動識別其內容，並給予建議:
        圖片是否模糊、太亮、太暗或有歪斜等問題，請給予具體中文建議。
        """
        with open(image_path, "rb") as image_file:
            image_content = image_file.read()
        encoded_image = base64.b64encode(image_content).decode('utf-8')

        response = model.generate_content(
            [
                {"mime_type": "application/octet-stream", "data": encoded_image},
                prompt,
            ]
        )

        if response.text:
            return {"error": response.text}

        return {"error": f"An error occurred: {e}"}

def recognize_image(image_path):
    try:
        # 讀取圖片
        with open(image_path, "rb") as image_file:
            image_content = image_file.read()

        image_type = imghdr.what(None, image_content)

        if image_type == 'jpeg':
            mime_type = 'image/jpeg'
        elif image_type == 'jpg':
            mime_type = 'image/jpeg'
        elif image_type == 'png':
            mime_type = 'image/png'
        elif image_type == 'heic':
            mime_type = 'image/heic'
        else:
            mime_type = 'application/octet-stream'

        encoded_image = encode_image(image_content)
        model = genai.GenerativeModel(model_name="gemini-1.5-pro")

        print(f"{os.path.basename(image_path)} has been loaded successfully.")

        # 先讓模型自動辨識圖片類別（vital_signs, medication, todo）
        prompt = """
        請根據圖片自動識別其內容，並以繁體中文輸出以下資訊：
        類別: [圖片類別]  # 可能的類別有: vital_signs(內容有可能是血壓計、健康檢查單、或是圖片等形式), medication(藥物), todo(代辦事項)

        """

        response = model.generate_content(
            [
                {"mime_type": mime_type, "data": encoded_image},
                prompt,
            ]
        )

        result = {}

        if response.text:
            # 根據模型回傳的內容進行解析
            category_match = re.search(r"類別:\s*(vital_signs|medication|todo)", response.text)
            info_match = re.search(r"相關資訊:\s*(.+)", response.text)

            category = category_match.group(1) if category_match else "未識別"
            info = info_match.group(1) if info_match else "未找到相關資訊"

            result["category"] = category
            result["data"] = info

            # 保留原來的程式邏輯不變

            if category == "vital_signs":

                prompt_vital = """
                請辨識圖片中的生命徵數讀數，並以繁體中文輸出以下資訊：
                高壓: [高壓值]
                低壓: [低壓值]
                心律: [心律值] # 請注意是否有❤的符號，若有就注意符號旁的數字並進行處理
                脈搏: [脈搏值]
                """
                response = model.generate_content(
                    [
                        {"mime_type": mime_type, "data": encoded_image},
                        prompt_vital,
                    ]
                )

                if response.text:
                    high_pressure = re.search(r"高壓:\s*(\d+)", response.text)
                    low_pressure = re.search(r"低壓:\s*(\d+)", response.text)
                    heart_rate = re.search(r"心律:\s*(\d+)", response.text)
                    pulse = re.search(r"脈搏:\s*(\d+)", response.text)

                    result["data"] = {
                        "high_pressure": high_pressure.group(1) if high_pressure else "未找到",
                        "low_pressure": low_pressure.group(1) if low_pressure else "未找到",
                        "heart_rate": heart_rate.group(1) if heart_rate else "未找到",
                        "pulse": pulse.group(1) if pulse else "未找到",
                    }
                else:
                    result["error"] = "No response text for vital signs."

            elif category == "medication":
              # 這裡處理藥物資料的邏輯
              prompt = """
              請辨識圖片中的所有藥物資訊，並以繁體中文輸出以下資訊：
              藥物名稱: [藥物名稱]
              用藥時間: [用藥時間 (HH:MM)]
              適應症: [適應症]
              副作用: [副作用]
              藥物顏色: [藥物顏色]
              藥物外觀: [藥物外觀]
              服藥警語: [服藥警語] # 服藥的注意事項(如果有)
              若有多個藥物，請分別輸出每個藥物的資訊。
              """
              response = model.generate_content(
                  [
                      {"mime_type": mime_type, "data": encoded_image},
                      prompt,
                  ]
              )

              if response.text:
                  medications = []
                  medication_data = response.text.split("\n\n")  # 假設每個藥品之間以空行分隔
                  valid_count = 0  # 記錄有效資料數量

                  # for data in medication_data:
                  for index, data in enumerate(medication_data, 1):
                      name = re.search(r"藥物名稱:\s*(.+)", data)
                      time = re.search(r"用藥時間:\s*(.+)", data)
                      indication = re.search(r"適應症:\s*(.+)", data)
                      side_effects = re.search(r"副作用:\s*(.+)", data)
                      color = re.search(r"藥物顏色:\s*(.+)", data)
                      appearance = re.search(r"藥物外觀:\s*(.+)", data)
                      warning = re.search(r"服藥警語:\s*(.+)", data)

                      medication = {
                          "name": name.group(1) if name else "未找到",
                          "time": time.group(1) if time else "未找到",
                          "indication": indication.group(1) if indication else "未找到",
                          "side_effects": side_effects.group(1) if side_effects else "未找到",
                          "color": color.group(1) if color else "未找到",
                          "appearance": appearance.group(1) if appearance else "未找到",
                          "warning": warning.group(1) if warning else "未找到",

                      }

                      # 檢查藥物資料是否有效
                      if not all(value == "未找到" for value in medication.values()):
                          medications.append(medication)
                          valid_count += 1  # 只增加有效資料

                  # 最終結果
                  result["category"] = category
                  result["data"] = medications  # 所有藥品的資訊列表
                  result["valid_count"] = valid_count  # 輸出有效資料筆數
              else:
                  result["error"] = "No response text."

            elif category == "todo":

                prompt_todo = """
                請辨識圖片中的待辦事項，並以繁體中文輸出以下資訊：
                事項: [事項內容]
                期限: [完成期限]
                優先級: [優先級]
                """
                response = model.generate_content(
                    [
                        {"mime_type": mime_type, "data": encoded_image},
                        prompt_todo,
                    ]
                )

                if response.text:
                    todos = []
                    todo_data = response.text.split("\n\n")
                    for data in todo_data:
                        item = re.search(r"事項:\s*(.+)", data)
                        deadline = re.search(r"期限:\s*(.+)", data)
                        priority = re.search(r"優先級:\s*(.+)", data)

                        todo = {
                            "item": item.group(1) if item else "未找到",
                            "deadline": deadline.group(1) if deadline else "未找到",
                            "priority": priority.group(1) if priority else "未找到",
                        }

                        todos.append(todo)

                    result["data"] = todos
                else:
                    result["error"] = "No response text for todo."

            else:
                result["error"] = "無法識別的資料類型。"

        else:
            result["error"] = "No response text from model."

        return result

    except FileNotFoundError:
        return {"error": f"Error: File not found at {image_path}"}
    except Exception as e:
        return {"error": f"An error occurred: {e}"}


def main():
    image_path = ""

    # Recognize image
    result = recognize_image(image_path)

    if "error" in result:
        print("Some error happened.")
        # Call error_type if recognition fails
        error_result = error_type(image_path)
        print(json.dumps(error_result, ensure_ascii=False, indent=4))

        # 請詢問是否需要重新辨識
        user_response = input("\n請確認資料是否正確? (確認/重新辨識): ")
        if user_response == "確認":
            print("資料已確認。")
        elif user_response == "重新辨識":
            print("重新辨識中...")
            main()
        else:
            print("無效的選項。請輸入 '確認' 或 '重新辨識'。")

    else:
        if result["category"] == "medication":
            # 藥物資訊需要編號
            print("\n辨識結果 (藥物資訊):")
            for i, medication in enumerate(result["data"], 1):
                print(f"藥物 {i}:")
                print(json.dumps(medication, ensure_ascii=False, indent=4))

            # 讓使用者選擇是否修改藥物資訊
            user_response = input("\n是否需要修改特定藥物資料? (修改/確認): ")
            if user_response == "修改":
                # 讓使用者選擇修改的藥物編號
                item_to_modify = int(input("\n請輸入藥物編號以修改 (例如輸入 1 修改藥物 1): "))
                if item_to_modify < 1 or item_to_modify > len(result["data"]):
                    print("無效的藥物編號。")
                else:
                    selected_medication = result["data"][item_to_modify - 1]  # 獲取選擇的藥物資料
                    print(f"\n選擇的藥物: {json.dumps(selected_medication, ensure_ascii=False, indent=4)}")

                    # 讓使用者修改藥物資料（整個藥品資料，而不是單獨修改特定欄位）
                    new_data = input(f"修改藥物資料 (當前: {json.dumps(selected_medication, ensure_ascii=False, indent=4)}): ")
                    if new_data:
                        try:
                            # 嘗試將輸入的資料解析為 JSON
                            updated_medication = json.loads(new_data)

                            # 確保所有必要欄位都存在，否則給予默認值
                            result["data"][item_to_modify - 1] = {
                                "name": updated_medication.get("name", selected_medication["name"]),
                                "time": updated_medication.get("time", selected_medication["time"]),
                                "indication": updated_medication.get("indication", selected_medication["indication"]),
                                "side_effects": updated_medication.get("side_effects", selected_medication["side_effects"]),
                                "color": updated_medication.get("color", selected_medication["color"]),
                                "appearance": updated_medication.get("appearance", selected_medication["appearance"]),
                                "warning": updated_medication.get("warning", selected_medication["warning"]),
                            }

                            print("\n藥物資料已更新：")
                            print(json.dumps(result["data"][item_to_modify - 1], ensure_ascii=False, indent=4))

                        except json.JSONDecodeError:
                            print("無效的資料格式，請重新輸入有效的 JSON 格式資料。")
                    else:
                        print("未做任何更改。")

            elif user_response == "確認":
                print("資料已確認。")
            else:
                print("無效的選項。請輸入 '修改' 或 '確認'。")

        elif result["category"] == "vital_signs":
            # 顯示vital_signs類別的結果
            print("\n辨識結果 (生命徵數):")
            print(json.dumps(result, ensure_ascii=False, indent=4))

            # 只有在類別為 vital_signs 時，顯示相關選項
            print("請選擇需要重新辨識的項目：")
            print("1. 高壓\n2. 低壓\n3. 心律\n4. 脈搏")

            selections = input("請輸入選擇的項目（以逗號分隔，例如 1,3）：")
            selected_fields = selections.split(",")
            # 在此處理每個選擇的欄位並重新辨識
            for field in selected_fields:
                print(f"重新辨識欄位: {field}")
                # 你可以在這裡根據選定的欄位進行重新辨識
                # 如：重新生成或處理特定的欄位

            # 確認重新辨識後，詢問是否重新確認資料
            user_response = input("\n請確認資料是否正確? (確認/重新辨識): ")
            if user_response == "確認":
                print("資料已確認。")
            elif user_response == "重新辨識":
                print("重新辨識中...")
                main()
            else:
                print("無效的選項。請輸入 '確認' 或 '重新辨識'。")

        elif result["category"] == "todo":
            # 顯示待辦事項類別的結果
            print("待辦事項資料如下：")
            print(json.dumps(result, ensure_ascii=False, indent=4))

            # 確認待辦事項資料後，詢問是否重新辨識
            user_response = input("\n請確認資料是否正確? (確認/重新辨識): ")
            if user_response == "確認":
                print("資料已確認。")
            elif user_response == "重新辨識":
                print("重新辨識中...")
                main()
            else:
                print("無效的選項。請輸入 '確認' 或 '重新辨識'。")

        else:
            # 其他類別默認 valid_count = 1
            result["valid_count"] = 1
            print(json.dumps(result, ensure_ascii=False, indent=4))

        # 重新確認結果
        user_response = input("\n請確認資料是否正確? (確認/重新辨識/選定欄位重新辨識): ")
        if user_response == "確認":
            print("資料已確認。")
        elif user_response == "重新辨識":
            print("重新辨識中...")
            main()
        else:
            print("無效的選項。請輸入 '確認' 或 '重新辨識' 或 '選定欄位重新辨識'。")


if __name__ == "__main__":
    main()


### 輸出格式錯誤:

是否需要修改特定藥物資料? (修改/確認): 修改

請輸入藥物編號以修改 (例如輸入 1 修改藥物 1): 1

---
應該要改成:
請確認資料是否正確? (確認/重新辨識/選定欄位重新辨識):


In [ ]:
def encode_image(image_content):
    """Encode image content to base64"""
    return base64.b64encode(image_content).decode('utf-8')

def is_valid_for_category(image_type, category):
    """Check if image type is valid for the selected category"""
    # Add validation logic here if needed
    # For now, we'll assume all image types are valid for all categories
    return True


def error_type(image_path):
    """Check for image quality issues and call Gemini for suggestions if needed"""
    try:
        # Open image using OpenCV
        image = cv2.imread(image_path)

        # Check brightness (average pixel value)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        brightness = np.mean(gray)
        if brightness < 50:
            return {"error": "Image too dark."}
        elif brightness > 200:
            return {"error": "Image too bright."}

        # Check for blurriness (using Laplacian variance)
        variance = cv2.Laplacian(gray, cv2.CV_64F).var()
        if variance < 100:
            return {"error": "Image is blurry."}

        # Check for orientation (detect edges and estimate rotation)
        # Using Hough Transform to check for line alignment
        edges = cv2.Canny(gray, 50, 150, apertureSize=3)
        lines = cv2.HoughLinesP(edges, 1, np.pi / 180, 100, minLineLength=50, maxLineGap=10)

        if lines is not None:
            angle = np.mean([np.arctan2(line[0][3] - line[0][1], line[0][2] - line[0][0]) for line in lines])
            angle = np.degrees(angle)
            if abs(angle) > 5:  # If the angle of the lines is greater than 5 degrees
                return {"error": "Image is tilted."}

        # If no issues, return None
        return {"error": "No issues detected with image quality."}

    except Exception as e:
        # If error during processing, call Gemini for suggestions
        model = genai.GenerativeModel(model_name="gemini-1.5-pro")
        prompt = """
        請根據圖片自動識別其內容，並給予建議:
        圖片是否模糊、太亮、太暗或有歪斜等問題，請給予具體中文建議。
        """
        with open(image_path, "rb") as image_file:
            image_content = image_file.read()
        encoded_image = base64.b64encode(image_content).decode('utf-8')

        response = model.generate_content(
            [
                {"mime_type": "application/octet-stream", "data": encoded_image},
                prompt,
            ]
        )

        if response.text:
            return {"error": response.text}

        return {"error": f"An error occurred: {e}"}

def recognize_image(image_path):
    try:
        # 讀取圖片
        with open(image_path, "rb") as image_file:
            image_content = image_file.read()

        image_type = imghdr.what(None, image_content)

        if image_type == 'jpeg':
            mime_type = 'image/jpeg'
        elif image_type == 'jpg':
            mime_type = 'image/jpeg'
        elif image_type == 'png':
            mime_type = 'image/png'
        elif image_type == 'heic':
            mime_type = 'image/heic'
        else:
            mime_type = 'application/octet-stream'

        encoded_image = encode_image(image_content)
        model = genai.GenerativeModel(model_name="gemini-1.5-pro")

        print(f"{os.path.basename(image_path)} has been loaded successfully.")

        # 先讓模型自動辨識圖片類別（vital_signs, medication, todo）
        prompt = """
        請根據圖片自動識別其內容，並以繁體中文輸出以下資訊：
        類別: [圖片類別]  # 可能的類別有: vital_signs(內容有可能是血壓計、健康檢查單、或是圖片等形式), medication(藥物), todo(代辦事項)

        """

        response = model.generate_content(
            [
                {"mime_type": mime_type, "data": encoded_image},
                prompt,
            ]
        )

        result = {}

        if response.text:
            # 根據模型回傳的內容進行解析
            category_match = re.search(r"類別:\s*(vital_signs|medication|todo)", response.text)
            info_match = re.search(r"相關資訊:\s*(.+)", response.text)

            category = category_match.group(1) if category_match else "未識別"
            info = info_match.group(1) if info_match else "未找到相關資訊"

            result["category"] = category
            result["data"] = info

            # 保留原來的程式邏輯不變

            if category == "vital_signs":

                prompt_vital = """
                請辨識圖片中的生命徵數讀數，並以繁體中文輸出以下資訊：
                高壓: [高壓值]
                低壓: [低壓值]
                心律: [心律值] # 請注意是否有❤的符號，若有就注意符號旁的數字並進行處理
                脈搏: [脈搏值]
                """
                response = model.generate_content(
                    [
                        {"mime_type": mime_type, "data": encoded_image},
                        prompt_vital,
                    ]
                )

                if response.text:
                    high_pressure = re.search(r"高壓:\s*(\d+)", response.text)
                    low_pressure = re.search(r"低壓:\s*(\d+)", response.text)
                    heart_rate = re.search(r"心律:\s*(\d+)", response.text)
                    pulse = re.search(r"脈搏:\s*(\d+)", response.text)

                    result["data"] = {
                        "high_pressure": high_pressure.group(1) if high_pressure else "未找到",
                        "low_pressure": low_pressure.group(1) if low_pressure else "未找到",
                        "heart_rate": heart_rate.group(1) if heart_rate else "未找到",
                        "pulse": pulse.group(1) if pulse else "未找到",
                    }
                else:
                    result["error"] = "No response text for vital signs."

            elif category == "medication":
              # 這裡處理藥物資料的邏輯
              prompt = """
              請辨識圖片中的所有藥物資訊，並以繁體中文輸出以下資訊：
              藥物名稱: [藥物名稱]
              用藥時間: [用藥時間 (HH:MM)]
              適應症: [適應症]
              副作用: [副作用]
              藥物顏色: [藥物顏色]
              藥物外觀: [藥物外觀]
              服藥警語: [服藥警語] # 服藥的注意事項(如果有)
              若有多個藥物，請分別輸出每個藥物的資訊。
              """
              response = model.generate_content(
                  [
                      {"mime_type": mime_type, "data": encoded_image},
                      prompt,
                  ]
              )

              if response.text:
                  medications = []
                  medication_data = response.text.split("\n\n")  # 假設每個藥品之間以空行分隔
                  valid_count = 0  # 記錄有效資料數量

                  # for data in medication_data:
                  for index, data in enumerate(medication_data, 1):
                      name = re.search(r"藥物名稱:\s*(.+)", data)
                      time = re.search(r"用藥時間:\s*(.+)", data)
                      indication = re.search(r"適應症:\s*(.+)", data)
                      side_effects = re.search(r"副作用:\s*(.+)", data)
                      color = re.search(r"藥物顏色:\s*(.+)", data)
                      appearance = re.search(r"藥物外觀:\s*(.+)", data)
                      warning = re.search(r"服藥警語:\s*(.+)", data)

                      medication = {
                          "name": name.group(1) if name else "未找到",
                          "time": time.group(1) if time else "未找到",
                          "indication": indication.group(1) if indication else "未找到",
                          "side_effects": side_effects.group(1) if side_effects else "未找到",
                          "color": color.group(1) if color else "未找到",
                          "appearance": appearance.group(1) if appearance else "未找到",
                          "warning": warning.group(1) if warning else "未找到",

                      }

                      # 檢查藥物資料是否有效
                      if not all(value == "未找到" for value in medication.values()):
                          medications.append(medication)
                          valid_count += 1  # 只增加有效資料

                  # 最終結果
                  result["category"] = category
                  result["data"] = medications  # 所有藥品的資訊列表
                  result["valid_count"] = valid_count  # 輸出有效資料筆數
              else:
                  result["error"] = "No response text."

            elif category == "todo":

                prompt_todo = """
                請辨識圖片中的待辦事項，並以繁體中文輸出以下資訊：
                事項: [事項內容]
                期限: [完成期限]
                優先級: [優先級]
                """
                response = model.generate_content(
                    [
                        {"mime_type": mime_type, "data": encoded_image},
                        prompt_todo,
                    ]
                )

                if response.text:
                    todos = []
                    todo_data = response.text.split("\n\n")
                    for data in todo_data:
                        item = re.search(r"事項:\s*(.+)", data)
                        deadline = re.search(r"期限:\s*(.+)", data)
                        priority = re.search(r"優先級:\s*(.+)", data)

                        todo = {
                            "item": item.group(1) if item else "未找到",
                            "deadline": deadline.group(1) if deadline else "未找到",
                            "priority": priority.group(1) if priority else "未找到",
                        }

                        todos.append(todo)

                    result["data"] = todos
                else:
                    result["error"] = "No response text for todo."

            else:
                result["error"] = "無法識別的資料類型。"

        else:
            result["error"] = "No response text from model."

        return result

    except FileNotFoundError:
        return {"error": f"Error: File not found at {image_path}"}
    except Exception as e:
        return {"error": f"An error occurred: {e}"}

import json

def check_image(result, category):
    if "error" in result:
        print("Some error happened.")
        error_result = error_type(result["image_path"])
        print(json.dumps(error_result, ensure_ascii=False, indent=4))

        user_response = input("\n請確認資料是否正確? (確認/重新辨識): ")
        if user_response == "確認":
            print("資料已確認。")
            return
        elif user_response == "重新辨識":
            print("重新辨識中...")
            main()
        else:
            print("無效的選項。請輸入 '確認' 或 '重新辨識'。")

    else:
        if category == "medication":
            print("\n辨識結果 (藥物資訊):")
            for i, medication in enumerate(result["data"], 1):
                print(f"藥物 {i}:")
                print(json.dumps(medication, ensure_ascii=False, indent=4))

            user_response = input("\n是否需要修改特定藥物資料? (修改/確認): ")
            if user_response == "修改":
                item_to_modify = int(input("\n請輸入藥物編號以修改 (例如輸入 1 修改藥物 1): "))
                if item_to_modify < 1 or item_to_modify > len(result["data"]):
                    print("無效的藥物編號。")
                else:
                    selected_medication = result["data"][item_to_modify - 1]
                    print(f"\n選擇的藥物: {json.dumps(selected_medication, ensure_ascii=False, indent=4)}")

                    new_data = input(f"修改藥物資料 (當前: {json.dumps(selected_medication, ensure_ascii=False, indent=4)}): ")
                    if new_data:
                        try:
                            updated_medication = json.loads(new_data)
                            result["data"][item_to_modify - 1] = {
                                "name": updated_medication.get("name", selected_medication["name"]),
                                "time": updated_medication.get("time", selected_medication["time"]),
                                "indication": updated_medication.get("indication", selected_medication["indication"]),
                                "side_effects": updated_medication.get("side_effects", selected_medication["side_effects"]),
                                "color": updated_medication.get("color", selected_medication["color"]),
                                "appearance": updated_medication.get("appearance", selected_medication["appearance"]),
                                "warning": updated_medication.get("warning", selected_medication["warning"]),
                            }
                            print("\n藥物資料已更新：")
                            print(json.dumps(result["data"][item_to_modify - 1], ensure_ascii=False, indent=4))
                        except json.JSONDecodeError:
                            print("無效的資料格式，請重新輸入有效的 JSON 格式資料。")
                    else:
                        print("未做任何更改。")

                print("\n請確認修改後的資料是否正確:")
                check_image(result, category)  # 加上提示句與空行

            elif user_response == "確認":
                print("資料已確認。")
                return
            else:
                print("無效的選項。請輸入 '修改' 或 '確認'。")

        elif category == "vital_signs":
            print("\n辨識結果 (生命徵數):")
            print(json.dumps(result, ensure_ascii=False, indent=4))

            user_response = input("\n請確認資料是否正確? (確認/重新辨識): ")
            if user_response == "確認":
                print("資料已確認。")
                return
            elif user_response == "重新辨識":
                print("重新辨識中...")
                main()
            else:
                print("無效的選項。請輸入 '確認' 或 '重新辨識'。")

        elif category == "todo":
            print("待辦事項資料如下：")
            print(json.dumps(result, ensure_ascii=False, indent=4))

            user_response = input("\n請確認資料是否正確? (確認/重新辨識): ")
            if user_response == "確認":
                print("資料已確認。")
                return
            elif user_response == "重新辨識":
                print("重新辨識中...")
                main()
            else:
                print("無效的選項。請輸入 '確認' 或 '重新辨識'。")

        else:
            result["valid_count"] = 1
            print(json.dumps(result, ensure_ascii=False, indent=4))

        user_response = input("\n請確認資料是否正確? (確認/重新辨識): ")
        if user_response == "確認":
            print("資料已確認。")
            return
        elif user_response == "重新辨識":
            print("重新辨識中...")
            main()
        else:
            print("無效的選項。請輸入 '確認' 或 '重新辨識'。")



def main():
    image_path = ""

    # Recognize image
    result = recognize_image(image_path)

    # 呼叫 check_image
    check_image(result, result["category"])

if __name__ == "__main__":
    main()


In [ ]:
def encode_image(image_content):
    """Encode image content to base64"""
    return base64.b64encode(image_content).decode('utf-8')

def is_valid_for_category(image_type, category):
    """Check if image type is valid for the selected category"""
    # Add validation logic here if needed
    # For now, we'll assume all image types are valid for all categories
    return True


def error_type(image_path):
    """Check for image quality issues and call Gemini for suggestions if needed"""
    try:
        # Open image using OpenCV
        image = cv2.imread(image_path)

        # Check brightness (average pixel value)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        brightness = np.mean(gray)
        if brightness < 50:
            return {"error": "Image too dark."}
        elif brightness > 200:
            return {"error": "Image too bright."}

        # Check for blurriness (using Laplacian variance)
        variance = cv2.Laplacian(gray, cv2.CV_64F).var()
        if variance < 100:
            return {"error": "Image is blurry."}

        # Check for orientation (detect edges and estimate rotation)
        # Using Hough Transform to check for line alignment
        edges = cv2.Canny(gray, 50, 150, apertureSize=3)
        lines = cv2.HoughLinesP(edges, 1, np.pi / 180, 100, minLineLength=50, maxLineGap=10)

        if lines is not None:
            angle = np.mean([np.arctan2(line[0][3] - line[0][1], line[0][2] - line[0][0]) for line in lines])
            angle = np.degrees(angle)
            if abs(angle) > 5:  # If the angle of the lines is greater than 5 degrees
                return {"error": "Image is tilted."}

        # If no issues, return None
        return {"error": "No issues detected with image quality."}

    except Exception as e:
        # If error during processing, call Gemini for suggestions
        model = genai.GenerativeModel(model_name="gemini-1.5-pro")
        prompt = """
        請根據圖片自動識別其內容，並給予建議:
        圖片是否模糊、太亮、太暗或有歪斜等問題，請給予具體中文建議。
        """
        with open(image_path, "rb") as image_file:
            image_content = image_file.read()
        encoded_image = base64.b64encode(image_content).decode('utf-8')

        response = model.generate_content(
            [
                {"mime_type": "application/octet-stream", "data": encoded_image},
                prompt,
            ]
        )

        if response.text:
            return {"error": response.text}

        return {"error": f"An error occurred: {e}"}

def recognize_image(image_path):
    try:
        # 讀取圖片
        with open(image_path, "rb") as image_file:
            image_content = image_file.read()

        image_type = imghdr.what(None, image_content)

        if image_type == 'jpeg':
            mime_type = 'image/jpeg'
        elif image_type == 'jpg':
            mime_type = 'image/jpeg'
        elif image_type == 'png':
            mime_type = 'image/png'
        elif image_type == 'heic':
            mime_type = 'image/heic'
        else:
            mime_type = 'application/octet-stream'

        encoded_image = encode_image(image_content)
        model = genai.GenerativeModel(model_name="gemini-1.5-pro")

        print(f"{os.path.basename(image_path)} has been loaded successfully.")

        # 先讓模型自動辨識圖片類別（vital_signs, medication, todo）
        prompt = """
        請根據圖片自動識別其內容，並以繁體中文輸出以下資訊：
        類別: [圖片類別]  # 可能的類別有: vital_signs(內容有可能是血壓計、健康檢查單、或是圖片等形式), medication(藥物), todo(代辦事項)

        """

        response = model.generate_content(
            [
                {"mime_type": mime_type, "data": encoded_image},
                prompt,
            ]
        )

        result = {}

        if response.text:
            # 根據模型回傳的內容進行解析
            category_match = re.search(r"類別:\s*(vital_signs|medication|todo)", response.text)
            info_match = re.search(r"相關資訊:\s*(.+)", response.text)

            category = category_match.group(1) if category_match else "未識別"
            info = info_match.group(1) if info_match else "未找到相關資訊"

            result["category"] = category
            result["data"] = info

            # 保留原來的程式邏輯不變

            if category == "vital_signs":

                prompt_vital = """
                請辨識圖片中的生命徵數讀數，並以繁體中文輸出以下資訊：
                高壓: [高壓值]
                低壓: [低壓值]
                心律: [心律值] # 請注意是否有❤的符號，若有就注意符號旁的數字並進行處理
                脈搏: [脈搏值]
                """
                response = model.generate_content(
                    [
                        {"mime_type": mime_type, "data": encoded_image},
                        prompt_vital,
                    ]
                )

                if response.text:
                    high_pressure = re.search(r"高壓:\s*(\d+)", response.text)
                    low_pressure = re.search(r"低壓:\s*(\d+)", response.text)
                    heart_rate = re.search(r"心律:\s*(\d+)", response.text)
                    pulse = re.search(r"脈搏:\s*(\d+)", response.text)

                    result["data"] = {
                        "high_pressure": high_pressure.group(1) if high_pressure else "未找到",
                        "low_pressure": low_pressure.group(1) if low_pressure else "未找到",
                        "heart_rate": heart_rate.group(1) if heart_rate else "未找到",
                        "pulse": pulse.group(1) if pulse else "未找到",
                    }
                else:
                    result["error"] = "No response text for vital signs."

            elif category == "medication":
              # 這裡處理藥物資料的邏輯
              prompt = """
              請辨識圖片中的所有藥物資訊，並以繁體中文輸出以下資訊：
              藥物名稱: [藥物名稱]
              用藥時間: [用藥時間 (HH:MM)]
              適應症: [適應症]
              副作用: [副作用]
              藥物顏色: [藥物顏色]
              藥物外觀: [藥物外觀]
              服藥警語: [服藥警語] # 服藥的注意事項(如果有)
              若有多個藥物，請分別輸出每個藥物的資訊。
              """
              response = model.generate_content(
                  [
                      {"mime_type": mime_type, "data": encoded_image},
                      prompt,
                  ]
              )

              if response.text:
                  medications = []
                  medication_data = response.text.split("\n\n")  # 假設每個藥品之間以空行分隔
                  valid_count = 0  # 記錄有效資料數量

                  # for data in medication_data:
                  for index, data in enumerate(medication_data, 1):
                      name = re.search(r"藥物名稱:\s*(.+)", data)
                      time = re.search(r"用藥時間:\s*(.+)", data)
                      indication = re.search(r"適應症:\s*(.+)", data)
                      side_effects = re.search(r"副作用:\s*(.+)", data)
                      color = re.search(r"藥物顏色:\s*(.+)", data)
                      appearance = re.search(r"藥物外觀:\s*(.+)", data)
                      warning = re.search(r"服藥警語:\s*(.+)", data)

                      medication = {
                          "name": name.group(1) if name else "未找到",
                          "time": time.group(1) if time else "未找到",
                          "indication": indication.group(1) if indication else "未找到",
                          "side_effects": side_effects.group(1) if side_effects else "未找到",
                          "color": color.group(1) if color else "未找到",
                          "appearance": appearance.group(1) if appearance else "未找到",
                          "warning": warning.group(1) if warning else "未找到",

                      }

                      # 檢查藥物資料是否有效
                      if not all(value == "未找到" for value in medication.values()):
                          medications.append(medication)
                          valid_count += 1  # 只增加有效資料

                  # 最終結果
                  result["category"] = category
                  result["data"] = medications  # 所有藥品的資訊列表
                  result["valid_count"] = valid_count  # 輸出有效資料筆數
              else:
                  result["error"] = "No response text."

            elif category == "todo":

                prompt_todo = """
                請辨識圖片中的待辦事項，並以繁體中文輸出以下資訊：
                事項: [事項內容]
                期限: [完成期限]
                優先級: [優先級]
                """
                response = model.generate_content(
                    [
                        {"mime_type": mime_type, "data": encoded_image},
                        prompt_todo,
                    ]
                )

                if response.text:
                    todos = []
                    todo_data = response.text.split("\n\n")
                    for data in todo_data:
                        item = re.search(r"事項:\s*(.+)", data)
                        deadline = re.search(r"期限:\s*(.+)", data)
                        priority = re.search(r"優先級:\s*(.+)", data)

                        todo = {
                            "item": item.group(1) if item else "未找到",
                            "deadline": deadline.group(1) if deadline else "未找到",
                            "priority": priority.group(1) if priority else "未找到",
                        }

                        todos.append(todo)

                    result["data"] = todos
                else:
                    result["error"] = "No response text for todo."

            else:
                result["error"] = "無法識別的資料類型。"

        else:
            result["error"] = "No response text from model."

        return result

    except FileNotFoundError:
        return {"error": f"Error: File not found at {image_path}"}
    except Exception as e:
        return {"error": f"An error occurred: {e}"}

def check_image(result, category):
    '''
    if "accuracy" in result:
            print(f"\n辨識準確率: {result['accuracy']}")
    '''

    # 如果有錯誤，處理錯誤
    if "error" in result:
        print("Some error happened.")
        error_result = error_type(result["image_path"])
        print(json.dumps(error_result, ensure_ascii=False, indent=4))

        user_response = input("\n請確認資料是否正確? (確認/重新辨識): ")
        if user_response == "確認":
            print("資料已確認。")
            return
        elif user_response == "重新辨識":
            print("重新辨識中...")
            main()
        else:
            print("無效的選項。請輸入 '確認' 或 '重新辨識'。")

    else:

        # 若有辨識率，檢查是否過低
        if "accuracy" in result and result["accuracy"] < 0.5:
            print("\n⚠️ 辨識率過低，請嘗試調整照片後重新辨識！")


        if category == "medication":
            print("\n辨識結果 (藥物資訊):")
            for i, medication in enumerate(result["data"], 1):
                print(f"藥物 {i}:")
                print(json.dumps(medication, ensure_ascii=False, indent=4))

            # 修改選項：重新辨識/修改特定藥物/確認
            user_response = input("\n是否需要修改藥物資料? (重新辨識/修改特定藥物/確認): ")

            if user_response == "重新辨識":
                print("重新辨識中...")
                main()

            elif user_response == "修改特定藥物":
                item_to_modify = int(input("\n請輸入藥物編號以修改 (例如輸入 1 修改藥物 1): "))
                if item_to_modify < 1 or item_to_modify > len(result["data"]):
                    print("無效的藥物編號。")
                else:
                    selected_medication = result["data"][item_to_modify - 1]
                    print(f"\n選擇的藥物: {json.dumps(selected_medication, ensure_ascii=False, indent=4)}")

                    new_data = input(f"修改藥物資料 (當前: {json.dumps(selected_medication, ensure_ascii=False, indent=4)}): ")
                    if new_data:
                        try:
                            updated_medication = json.loads(new_data)
                            result["data"][item_to_modify - 1] = {
                                "name": updated_medication.get("name", selected_medication["name"]),
                                "time": updated_medication.get("time", selected_medication["time"]),
                                "indication": updated_medication.get("indication", selected_medication["indication"]),
                                "side_effects": updated_medication.get("side_effects", selected_medication["side_effects"]),
                                "color": updated_medication.get("color", selected_medication["color"]),
                                "appearance": updated_medication.get("appearance", selected_medication["appearance"]),
                                "warning": updated_medication.get("warning", selected_medication["warning"]),
                            }
                            print("\n藥物資料已更新：")
                            print(json.dumps(result["data"][item_to_modify - 1], ensure_ascii=False, indent=4))
                        except json.JSONDecodeError:
                            print("無效的資料格式，請重新輸入有效的 JSON 格式資料。")
                    else:
                        print("未做任何更改。")

                print("\n請確認修改後的資料是否正確:")
                check_image(result, category)

            elif user_response == "確認":
                print("資料已確認。")
                return
            else:
                print("無效的選項。請輸入 '重新辨識' 或 '修改特定藥物' 或 '確認'。")

        elif category == "vital_signs":
            print("\n辨識結果 (生命徵數):")
            print(json.dumps(result, ensure_ascii=False, indent=4))

            user_response = input("\n請確認資料是否正確? (確認/重新辨識): ")
            if user_response == "確認":
                print("資料已確認。")
                return
            elif user_response == "重新辨識":
                print("重新辨識中...")
                main()
            else:
                print("無效的選項。請輸入 '確認' 或 '重新辨識'。")

        elif category == "todo":
            print("待辦事項資料如下：")
            print(json.dumps(result, ensure_ascii=False, indent=4))

            user_response = input("\n請確認資料是否正確? (確認/重新辨識): ")
            if user_response == "確認":
                print("資料已確認。")
                return
            elif user_response == "重新辨識":
                print("重新辨識中...")
                main()
            else:
                print("無效的選項。請輸入 '確認' 或 '重新辨識'。")

        else:
            result["valid_count"] = 1
            print(json.dumps(result, ensure_ascii=False, indent=4))

        user_response = input("\n請確認資料是否正確? (確認/重新辨識): ")
        if user_response == "確認":
            print("資料已確認。")
            return
        elif user_response == "重新辨識":
            print("重新辨識中...")
            main()
        else:
            print("無效的選項。請輸入 '確認' 或 '重新辨識'。")

def main():
    image_path = ""

    # Recognize image
    result = recognize_image(image_path)


    # 呼叫 check_image
    check_image(result, result["category"])

if __name__ == "__main__":
    main()


### OpenCV

In [ ]:
# !pip install opencv-python

In [ ]:
!pip uninstall opencv-python opencv-contrib-python opencv-python-headless -y
!pip install opencv-contrib-python==4.11.0.86

In [ ]:
import cv2
print(cv2.__version__)
print(dir(cv2.dnn_superres))

In [ ]:
def encode_image(image_content):
    """Encode image content to base64"""
    return base64.b64encode(image_content).decode('utf-8')

def is_valid_for_category(image_type, category):
    """Check if image type is valid for the selected category"""
    # Add validation logic here if needed
    # For now, we'll assume all image types are valid for all categories
    return True


def error_type(image_path):
    """Check for image quality issues and call Gemini for suggestions if needed"""
    try:
        # Open image using OpenCV
        image = cv2.imread(image_path)

        # Check brightness (average pixel value)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        brightness = np.mean(gray)
        if brightness < 50:
            return {"error": "Image too dark."}
        elif brightness > 200:
            return {"error": "Image too bright."}

        # Check for blurriness (using Laplacian variance)
        variance = cv2.Laplacian(gray, cv2.CV_64F).var()
        if variance < 100:
            return {"error": "Image is blurry."}

        # Check for orientation (detect edges and estimate rotation)
        # Using Hough Transform to check for line alignment
        edges = cv2.Canny(gray, 50, 150, apertureSize=3)
        lines = cv2.HoughLinesP(edges, 1, np.pi / 180, 100, minLineLength=50, maxLineGap=10)

        if lines is not None:
            angle = np.mean([np.arctan2(line[0][3] - line[0][1], line[0][2] - line[0][0]) for line in lines])
            angle = np.degrees(angle)
            if abs(angle) > 5:  # If the angle of the lines is greater than 5 degrees
                return {"error": "Image is tilted."}

        # If no issues, return None
        return {"error": "No issues detected with image quality."}

    except Exception as e:
        # If error during processing, call Gemini for suggestions
        model = genai.GenerativeModel(model_name="gemini-1.5-pro")
        prompt = """
        請根據圖片自動識別其內容，並給予建議:
        圖片是否模糊、太亮、太暗或有歪斜等問題，請給予具體中文建議。
        """
        with open(image_path, "rb") as image_file:
            image_content = image_file.read()
        encoded_image = base64.b64encode(image_content).decode('utf-8')

        response = model.generate_content(
            [
                {"mime_type": "application/octet-stream", "data": encoded_image},
                prompt,
            ]
        )

        if response.text:
            return {"error": response.text}

        return {"error": f"An error occurred: {e}"}

def recognize_image(image_path):
    try:
        # 讀取圖片
        with open(image_path, "rb") as image_file:
            image_content = image_file.read()

        image_type = imghdr.what(None, image_content)

        if image_type == 'jpeg':
            mime_type = 'image/jpeg'
        elif image_type == 'jpg':
            mime_type = 'image/jpeg'
        elif image_type == 'png':
            mime_type = 'image/png'
        elif image_type == 'heic':
            mime_type = 'image/heic'
        else:
            mime_type = 'application/octet-stream'

        encoded_image = encode_image(image_content)
        model = genai.GenerativeModel(model_name="gemini-1.5-pro")

        print(f"{os.path.basename(image_path)} has been loaded successfully.")

        # 先讓模型自動辨識圖片類別（vital_signs, medication, todo）
        prompt = """
        請根據圖片自動識別其內容，並以繁體中文輸出以下資訊：
        類別: [圖片類別]  # 可能的類別有: vital_signs(內容有可能是血壓計、健康檢查單、或是圖片等形式), medication(藥物), todo(代辦事項)

        """

        response = model.generate_content(
            [
                {"mime_type": mime_type, "data": encoded_image},
                prompt,
            ]
        )

        result = {}

        if response.text:
            # 根據模型回傳的內容進行解析
            category_match = re.search(r"類別:\s*(vital_signs|medication|todo)", response.text)
            info_match = re.search(r"相關資訊:\s*(.+)", response.text)

            category = category_match.group(1) if category_match else "未識別"
            info = info_match.group(1) if info_match else "未找到相關資訊"

            result["category"] = category
            result["data"] = info

            # 保留原來的程式邏輯不變

            if category == "vital_signs":

                prompt_vital = """
                請辨識圖片中的生命徵數讀數，並以繁體中文輸出以下資訊：
                高壓: [高壓值]
                低壓: [低壓值]
                心律: [心律值] # 請注意是否有❤的符號，若有就注意符號旁的數字並進行處理
                脈搏: [脈搏值]
                """
                response = model.generate_content(
                    [
                        {"mime_type": mime_type, "data": encoded_image},
                        prompt_vital,
                    ]
                )

                if response.text:
                    high_pressure = re.search(r"高壓:\s*(\d+)", response.text)
                    low_pressure = re.search(r"低壓:\s*(\d+)", response.text)
                    heart_rate = re.search(r"心律:\s*(\d+)", response.text)
                    pulse = re.search(r"脈搏:\s*(\d+)", response.text)

                    result["data"] = {
                        "high_pressure": high_pressure.group(1) if high_pressure else "未找到",
                        "low_pressure": low_pressure.group(1) if low_pressure else "未找到",
                        "heart_rate": heart_rate.group(1) if heart_rate else "未找到",
                        "pulse": pulse.group(1) if pulse else "未找到",
                    }
                else:
                    result["error"] = "No response text for vital signs."

            elif category == "medication":
              # 這裡處理藥物資料的邏輯
              prompt = """
              請辨識圖片中的所有藥物資訊，並以繁體中文輸出以下資訊：
              藥物名稱: [藥物名稱]
              用藥時間: [用藥時間 (HH:MM)]
              適應症: [適應症]
              副作用: [副作用]
              藥物顏色: [藥物顏色]
              藥物外觀: [藥物外觀]
              服藥警語: [服藥警語] # 服藥的注意事項(如果有)
              若有多個藥物，請分別輸出每個藥物的資訊。
              """
              response = model.generate_content(
                  [
                      {"mime_type": mime_type, "data": encoded_image},
                      prompt,
                  ]
              )

              if response.text:
                  medications = []
                  medication_data = response.text.split("\n\n")  # 假設每個藥品之間以空行分隔
                  valid_count = 0  # 記錄有效資料數量

                  # for data in medication_data:
                  for index, data in enumerate(medication_data, 1):
                      name = re.search(r"藥物名稱:\s*(.+)", data)
                      time = re.search(r"用藥時間:\s*(.+)", data)
                      indication = re.search(r"適應症:\s*(.+)", data)
                      side_effects = re.search(r"副作用:\s*(.+)", data)
                      color = re.search(r"藥物顏色:\s*(.+)", data)
                      appearance = re.search(r"藥物外觀:\s*(.+)", data)
                      warning = re.search(r"服藥警語:\s*(.+)", data)

                      medication = {
                          "name": name.group(1) if name else "未找到",
                          "time": time.group(1) if time else "未找到",
                          "indication": indication.group(1) if indication else "未找到",
                          "side_effects": side_effects.group(1) if side_effects else "未找到",
                          "color": color.group(1) if color else "未找到",
                          "appearance": appearance.group(1) if appearance else "未找到",
                          "warning": warning.group(1) if warning else "未找到",

                      }

                      # 檢查藥物資料是否有效
                      if not all(value == "未找到" for value in medication.values()):
                          medications.append(medication)
                          valid_count += 1  # 只增加有效資料

                  # 最終結果
                  result["category"] = category
                  result["data"] = medications  # 所有藥品的資訊列表
                  result["valid_count"] = valid_count  # 輸出有效資料筆數
              else:
                  result["error"] = "No response text."

            elif category == "todo":

                prompt_todo = """
                請辨識圖片中的待辦事項，並以繁體中文輸出以下資訊：
                事項: [事項內容]
                期限: [完成期限]
                優先級: [優先級]
                """
                response = model.generate_content(
                    [
                        {"mime_type": mime_type, "data": encoded_image},
                        prompt_todo,
                    ]
                )

                if response.text:
                    todos = []
                    todo_data = response.text.split("\n\n")
                    for data in todo_data:
                        item = re.search(r"事項:\s*(.+)", data)
                        deadline = re.search(r"期限:\s*(.+)", data)
                        priority = re.search(r"優先級:\s*(.+)", data)

                        todo = {
                            "item": item.group(1) if item else "未找到",
                            "deadline": deadline.group(1) if deadline else "未找到",
                            "priority": priority.group(1) if priority else "未找到",
                        }

                        todos.append(todo)

                    result["data"] = todos
                else:
                    result["error"] = "No response text for todo."

            else:
                result["error"] = "無法識別的資料類型。"

        else:
            result["error"] = "No response text from model."

        return result

    except FileNotFoundError:
        return {"error": f"Error: File not found at {image_path}"}
    except Exception as e:
        return {"error": f"An error occurred: {e}"}

def check_image(result, category):
    # 如果有錯誤，處理錯誤
    if "error" in result:
        print("Some error happened.")
        error_result = error_type(result["image_path"])
        print(json.dumps(error_result, ensure_ascii=False, indent=4))

        user_response = input("\n請確認資料是否正確? (確認/重新辨識): ")
        if user_response == "確認":
            print("資料已確認。")
            return
        elif user_response == "重新辨識":
            print("重新辨識中...")
            main()
        else:
            print("無效的選項。請輸入 '確認' 或 '重新辨識'。")

    else:
        # 若有辨識率，檢查是否過低
        if "accuracy" in result and result["accuracy"] < 0.7:
            print("\n⚠️ 辨識率過低，請嘗試調整照片後重新辨識！")

        if category == "medication":
            print("\n辨識結果 (藥物資訊):")
            for i, medication in enumerate(result["data"], 1):
                print(f"藥物 {i}:")
                print(json.dumps(medication, ensure_ascii=False, indent=4))

            # 修改選項：重新辨識/修改特定藥物/確認
            user_response = input("\n是否需要修改藥物資料? (重新辨識/修改特定藥物/確認): ")

            if user_response == "重新辨識":
                print("重新辨識中...")
                main()

            elif user_response == "修改特定藥物":
                item_to_modify = int(input("\n請輸入藥物編號以修改 (例如輸入 1 修改藥物 1): "))
                if item_to_modify < 1 or item_to_modify > len(result["data"]):
                    print("無效的藥物編號。")
                else:
                    selected_medication = result["data"][item_to_modify - 1]
                    print(f"\n選擇的藥物: {json.dumps(selected_medication, ensure_ascii=False, indent=4)}")

                    new_data = input(f"修改藥物資料 (當前: {json.dumps(selected_medication, ensure_ascii=False, indent=4)}): ")
                    if new_data:
                        try:
                            updated_medication = json.loads(new_data)
                            result["data"][item_to_modify - 1] = {
                                "name": updated_medication.get("name", selected_medication["name"]),
                                "time": updated_medication.get("time", selected_medication["time"]),
                                "indication": updated_medication.get("indication", selected_medication["indication"]),
                                "side_effects": updated_medication.get("side_effects", selected_medication["side_effects"]),
                                "color": updated_medication.get("color", selected_medication["color"]),
                                "appearance": updated_medication.get("appearance", selected_medication["appearance"]),
                                "warning": updated_medication.get("warning", selected_medication["warning"]),
                            }
                            print("\n藥物資料已更新：")
                            print(json.dumps(result["data"][item_to_modify - 1], ensure_ascii=False, indent=4))
                        except json.JSONDecodeError:
                            print("無效的資料格式，請重新輸入有效的 JSON 格式資料。")
                    else:
                        print("未做任何更改。")

                print("\n請確認修改後的資料是否正確:")
                check_image(result, category)

            elif user_response == "確認":
                print("資料已確認。")
                return
            else:
                print("無效的選項。請輸入 '重新辨識' 或 '修改特定藥物' 或 '確認'。")

        elif category == "vital_signs":
            print("\n辨識結果 (生命徵數):")
            print(json.dumps(result, ensure_ascii=False, indent=4))

            user_response = input("\n請確認資料是否正確? (確認/重新辨識): ")
            if user_response == "確認":
                print("資料已確認。")
                return
            elif user_response == "重新辨識":
                print("重新辨識中...")
                main()
            else:
                print("無效的選項。請輸入 '確認' 或 '重新辨識'。")

        elif category == "todo":
            print("待辦事項資料如下：")
            print(json.dumps(result, ensure_ascii=False, indent=4))

            user_response = input("\n請確認資料是否正確? (確認/重新辨識): ")
            if user_response == "確認":
                print("資料已確認。")
                return
            elif user_response == "重新辨識":
                print("重新辨識中...")
                main()
            else:
                print("無效的選項。請輸入 '確認' 或 '重新辨識'。")

        else:
            result["valid_count"] = 1
            print(json.dumps(result, ensure_ascii=False, indent=4))

        user_response = input("\n請確認資料是否正確? (確認/重新辨識): ")
        if user_response == "確認":
            print("資料已確認。")
            return
        elif user_response == "重新辨識":
            print("重新辨識中...")
            main()
        else:
            print("無效的選項。請輸入 '確認' 或 '重新辨識'。")

def enhance_image(image_path):
    """Enhance image using OpenCV's super-resolution methods"""
    # 讀取影像
    image = cv2.imread(image_path)

    # 使用 OpenCV 的 DnnSuperRes 進行超解析度
    sr = cv2.dnn_superres.DnnSuperResImpl_create()

    # 載入預訓練模型（例如 EDSR 模型，這裡以 4x 超解析度為例）
    sr.readModel("EDSR_x4.pb")  # 確保你下載並放置模型檔案在工作目錄中
    sr.setModel("edsr", 4)

    # 使用超解析度放大影像
    enhanced_image = sr.upsample(image)

    # 儲存或顯示增強後的影像
    enhanced_image_path = "enhanced_image.jpg"
    cv2.imwrite(enhanced_image_path, enhanced_image)

    print(f"增強後的影像已儲存為 {enhanced_image_path}")
    return enhanced_image_path

def main():
    image_path = "/content/drive/My Drive/Colab Notebooks/CP_im_re/PictureSet/error3.JPG"

    # 在開始辨識之前，先進行影像增強處理
    enhanced_image_path = enhance_image(image_path)

    # Recognize image
    result = recognize_image(enhanced_image_path)

    # 呼叫 check_image
    check_image(result, result["category"])

if __name__ == "__main__":
    main()


### 使用 Real-ESRGAN 進行影像超解析增強，修復模糊影像。

In [ ]:
!pip install basicsr
!pip install facexlib
!pip install gfpgan
!pip install -U realesrgan

In [ ]:
!wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -P weights

In [ ]:
!pip install torchvision --upgrade
!pip install realesrgan

In [ ]:
from realesrgan import RealESRGAN

In [ ]:
def repair(image_path, output_path="enhanced_image.png"):
    """
    使用 Real-ESRGAN 進行影像超解析增強，修復模糊影像。

    參數:
    - image_path: str, 原始模糊影像的路徑
    - output_path: str, 儲存增強後影像的路徑 (預設: enhanced_image.png)

    回傳:
    - 增強後的影像
    """
    try:
        # 讀取圖片
        img = Image.open(image_path)

        # 載入 Real-ESRGAN 模型 (x4 超解析)
        model = RealESRGAN('weights/RealESRGAN_x4plus.pth')
        model.load_model()

        # 進行影像增強
        enhanced_img = model.enhance(img)

        # 儲存增強後影像
        enhanced_img.save(output_path)
        print(f"✅ 影像增強完成，已儲存至 {output_path}")

        return enhanced_img

    except Exception as e:
        print(f"❌ 修復失敗: {e}")
        return None
